# Training on Keras Tensorflow

In [21]:
import glob
import librosa
import librosa.display
import numpy as np
import numpy
import _pickle as pickle
from sklearn import svm
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestNeighbors

In [22]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import Model
from keras.utils import plot_model
from keras.optimizers import SGD, Adadelta, Adagrad

In [25]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
                                              sr=sample_rate).T, axis=0)
    return mfccs, chroma, mel, contrast, tonnetz


def parse_audio_files(path):
    features, labels = np.empty((0, 193)), np.empty(0)
    labels = []
    for fn in glob.glob(path):
        try:
            mfccs, chroma, mel, contrast, tonnetz = extract_feature(fn)
        except Exception as e:
            print("Error encountered while parsing file: ", fn)
            continue
        ext_features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
        features = np.vstack([features, ext_features])
        labels = np.append(labels, fn.split("_")[4].split(".")[0])
        print(fn)
    return np.array(features), np.array(labels)


tr_features, tr_labels = parse_audio_files('./train_sounds_3/*.wav')

print(tr_labels)

tr_features = np.array(tr_features, dtype=pd.Series)
tr_labels = np.array(tr_labels, dtype=pd.Series)

X=tr_features.astype(int)
Y=tr_labels.astype(str)
seed = 7
numpy.random.seed(seed)

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)



# define baseline model
def baseline_model():
    model = Sequential()
    model.add(Dense(8, input_dim=193, activation='relu'))
    model.add(Dense(7, activation='softmax'))
# Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='binary_crossentropy', optimizer=sgd)
estimator = KerasClassifier(model, epochs=200, batch_size=5, verbose=0) #KerasClassifier
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))




filename = 'baseline_model_protocol_keras.sav'

pickle.dump(model, open(filename, 'wb'), protocol=2)

print('Model Saved..')


# print('Score:', estimator.score(X=tr_features.astype(int), y=tr_labels.astype(str)))

./train_sounds_3\OAF_back_angry.wav
./train_sounds_3\OAF_back_disgust.wav
./train_sounds_3\OAF_back_fear.wav
./train_sounds_3\OAF_back_happy.wav
./train_sounds_3\OAF_back_neutral.wav
./train_sounds_3\OAF_back_ps.wav
./train_sounds_3\OAF_back_sad.wav
./train_sounds_3\OAF_bar_angry.wav
./train_sounds_3\OAF_bar_disgust.wav
./train_sounds_3\OAF_bar_fear.wav
./train_sounds_3\OAF_bar_happy.wav
./train_sounds_3\OAF_bar_neutral.wav
./train_sounds_3\OAF_bar_ps.wav
./train_sounds_3\OAF_bar_sad.wav
./train_sounds_3\OAF_base_angry.wav
./train_sounds_3\OAF_base_disgust.wav
./train_sounds_3\OAF_base_fear.wav
./train_sounds_3\OAF_base_happy.wav
./train_sounds_3\OAF_base_neutral.wav
./train_sounds_3\OAF_base_ps.wav
./train_sounds_3\OAF_base_sad.wav
./train_sounds_3\OAF_bath_angry.wav
./train_sounds_3\OAF_bath_disgust.wav
./train_sounds_3\OAF_bath_fear.wav
./train_sounds_3\OAF_bath_happy.wav
./train_sounds_3\OAF_bath_neutral.wav
./train_sounds_3\OAF_bath_ps.wav
./train_sounds_3\OA_bite_neutral.wav
['an

AttributeError: 'NoneType' object has no attribute 'update'

# Testing the model

In [ ]:
import glob
import librosa
import librosa.display
import numpy as np
import _pickle as pickle
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn

In [ ]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import Model

In [ ]:
# Method to extract features from speech using librosa


def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
                                              sr=sample_rate).T, axis=0)
    return mfccs, chroma, mel, contrast, tonnetz


# List to store file names
target_files = []

# Method to extract label name and extract features from audio file


def parse_audio_files(path):
    labels = []
    features = np.empty((0, 193))
    for fn in glob.glob(path):
        try:
            mfccs, chroma, mel, contrast, tonnetz = extract_feature(fn)
        except Exception as e:
            print("Error encountered while parsing file: ", fn)
            continue
        ext_features = np.hstack([mfccs, chroma, mel, contrast, tonnetz])
        features = np.vstack([features, ext_features])
        labels = np.append(labels, fn.split("_")[4].split(".")[0])
        target_files.append(fn)
    return np.array(features), np.array(labels)


# Get labels and features of audion file of specified path
ts_features, ts_labels = parse_audio_files('./test_sounds_3/*.wav')

# Convert features and labels to the pandas Series data type
ts_features = np.array(ts_features, dtype=pd.Series)
ts_labels = np.array(ts_labels, dtype=pd.Series)

# Trained Model file name
filename = 'baseline_model_protocol_keras.sav'

# Load saved model from file
model = pickle.load(open(filename, 'rb'))

# Predict the emotion class
prediction = model.KerasClassifier(ts_features)

# True labels
test_true = ts_labels

# List to store prediction
test_predicted = []

# Iterate over model prediction and store it into list
for i, val in enumerate(prediction):
    test_predicted.append(val)


# Accuracy score of model
print('Accuracy Score:', accuracy_score(test_true, test_predicted))

# Number of corrected prediction
print('Number of correct prediction:', accuracy_score(test_true, test_predicted, normalize=False), 'out of', len(ts_labels))